In [ ]:
from google.colab import files
from tqdm import tqdm
# uploaded = files.upload()
import gcsfs
import os
import pandas as pd
import glob
import dask.dataframe as dd
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from dask.diagnostics import ProgressBar
import numpy as np
import networkx as nx
import itertools
import random

In [ ]:
# adding a progress bar for every code run using dask
ProgressBar().register()

In [ ]:
# uploaded = files.upload()
# making a filesystem fs to download the parquet data files
fs = gcsfs.GCSFileSystem(project='zippy-avatar-454716-s7',
                         token='zippy-avatar-454716-s7-6ce326d91b28.json')

In [ ]:
# to check the totol number of files in the gcp bucket
files1 = fs.ls('github-networks-bucket/feb_apr_2019_export')
files2 = fs.ls('github-networks-bucket/feb_apr_2020_export')
print(f"Total files: {len(files1)}")
print(f"Total files: {len(files2)}")
# print(files[:5])  # Show a few filenames

Total files: 2500
Total files: 5000


In [ ]:
# checking the data for the 1st file
with fs.open(files1[0]) as f:
    df = pd.read_parquet(f)

df.head()

,id,type,created_at,actor,repo,payload
0,9467373391,IssuesEvent,2019-04-18 08:08:45,kuntalacharyya,01org/isa-l,"{""action"":""closed"",""issue"":{""url"":""https://api..."
1,9163166325,IssuesEvent,2019-03-01 15:16:55,GUI,18F/api.data.gov,"{""action"":""closed"",""issue"":{""url"":""https://api..."
2,9513437118,IssuesEvent,2019-04-25 20:41:28,fureigh,18F/federal-grant-reporting,"{""action"":""closed"",""issue"":{""url"":""https://api..."
3,9425887876,IssuesEvent,2019-04-11 17:10:01,soyeb-jim285,3b1b/manim,"{""action"":""closed"",""issue"":{""url"":""https://api..."
4,9025313667,IssuesEvent,2019-02-07 13:07:32,ciarmcom,ARMmbed/mbed-os-5-docs,"{""action"":""closed"",""issue"":{""url"":""https://api..."


In [ ]:
# making a directory
local_dir_2019 = "/content/feb_apr_2019_export"
local_dir_2020 = "/content/feb_apr_2020_export"

os.makedirs(local_dir_2019, exist_ok=True)
os.makedirs(local_dir_2020, exist_ok=True)

In [ ]:
# downloading and storing the data
for file in tqdm(files1, desc = "Downloading 2019 files", unit = "file"):
  local_path = os.path.join(local_dir_2019, os.path.basename(file))
  with fs.open(file, 'rb') as fsrc, open(local_path, 'wb') as fdst:
    fdst.write(fsrc.read())

for file in tqdm(files2, desc = "Downloading 2020 files", unit = "file"):
  local_path = os.path.join(local_dir_2020, os.path.basename(file))
  with fs.open(file, 'rb') as fsrc, open(local_path, 'wb') as fdst:
    fdst.write(fsrc.read())

In [ ]:
parquet_files1 = glob.glob("/content/feb_apr_2019_export/*.parquet")
parquet_files2 = glob.glob("/content/feb_apr_2020_export/*.parquet")

print(len(parquet_files1 + parquet_files2))

7500


In [ ]:
# Using dask to create a lazy df
df_feb_apr_19 = dd.read_parquet("/content/feb_apr_2019_export/*.parquet")
df_feb_apr_20 = dd.read_parquet("/content/feb_apr_2020_export/*.parquet")

In [ ]:
df_feb_apr_19.head()
df_feb_apr_20.head()